# 🧮 PyKwant Tutorial: Numerical Methods
This notebook introduces `pykwant.numerics`, the foundational module providing essential numerical algorithms for quantitative finance: interpolation, numerical differentiation, and root finding.

Consistent with the library's design, this module relies heavily on **Higher-Order Functions**: functions that accept other functions as arguments or return them as results.

## 1. Setup and Imports

In [1]:
import math
from pykwant import numerics

# Helper to print results clearly
def print_result(label: str, value: float):
    print(f"{label}: {value:.6f}")

## 2. Interpolation
Interpolation is crucial for constructing curves (e.g., yield curves) from discrete data points. `pykwant` provides factory functions that return an `Interpolator` (a callable).

### Linear Interpolation
The `linear_interpolation` function creates a closure that captures the sorted data points and performs linear interpolation between them.

In [2]:
# Sample Data: Time (years) vs Rate (%)
x_data = [0.0, 1.0, 2.0,  5.0]
y_data = [2.0, 2.5, 3.0, 4.5]

# Create the interpolator function
# This returns a function `interp(x) -> y`
linear_interp = numerics.linear_interpolation(x_data, y_data)

print("--- Linear Interpolation ---")
# Interpolating between known points (x=1.5 is between 1.0 and 2.0)
y_1_5 = linear_interp(1.5)
print_result("f(1.5)", y_1_5) # Expected: 2.75 (midpoint of 2.5 and 3.0)

# Extrapolation (enabled by default)
y_6 = linear_interp(6.0)
print_result("f(6.0) [Extrapolated]", y_6)

--- Linear Interpolation ---
f(1.5): 2.750000
f(6.0) [Extrapolated]: 4.500000


### Controlling Extrapolation
We can disable extrapolation. If we query a point outside the domain, it returns `NaN`.

In [3]:
# Create an interpolator that strictly forbids extrapolation
strict_interp = numerics.linear_interpolation(x_data, y_data, extrapolate=False)

print("\n--- Strict Interpolation (No Extrapolation) ---")
print(f"f(0.5): {strict_interp(0.5)}") # OK
print(f"f(6.0): {strict_interp(6.0)}") # Returns nan


--- Strict Interpolation (No Extrapolation) ---
f(0.5): 2.25
f(6.0): nan


### Log-Linear Interpolation
Financial data like **Discount Factors** often behave exponentially. Linearly interpolating them can lead to arbitrage opportunities. `log_linear_interpolation` transforms $y$ to $\ln(y)$, interpolates linearly, and maps back with $\exp$.

In [4]:
# Data representing exponential decay (e.g., Discount Factors)
# e^-0 = 1, e^-1 ~ 0.367, e^-2 ~ 0.135
x_exp = [0.0, 1.0, 2.0]
y_exp = [math.exp(-x) for x in x_exp] 

# Compare Linear vs Log-Linear at x=0.5
lin_interp = numerics.linear_interpolation(x_exp, y_exp)
log_interp = numerics.log_linear_interpolation(x_exp, y_exp)

expected = math.exp(-0.5)

print("\n--- Linear vs Log-Linear (Target: exp(-0.5)) ---")
print_result("True Value", expected)
print_result("Linear    ", lin_interp(0.5))      # Overestimates slightly
print_result("Log-Linear", log_interp(0.5))      # Exact for exponential data


--- Linear vs Log-Linear (Target: exp(-0.5)) ---
True Value: 0.606531
Linear    : 0.683940
Log-Linear: 0.606531


## 3. Numerical Differentiation
The `numerical_derivative` function is a classic higher-order function. It takes a function $f$ and returns its approximate derivative function $f'$ using the central difference method: $f'(x) \approx \frac{f(x+h) - f(x-h)}{2h}$.

In [5]:
def cubic(x: float) -> float:
    return x**3

# Create the derivative function dynamically
# Analytical derivative: 3x^2
deriv_cubic = numerics.numerical_derivative(cubic, h=1e-5)

x_val = 2.0
print("\n--- Numerical Differentiation of x^3 at x=2.0 ---")
print_result("Approximation", deriv_cubic(x_val))
print_result("Analytical   ", 3 * x_val**2)


--- Numerical Differentiation of x^3 at x=2.0 ---
Approximation: 12.000000
Analytical   : 12.000000


## 4. Root Finding (Newton-Raphson)
`newton_solve` finds $x$ such that $f(x) = \text{target}$. It internally uses `numerical_derivative` to compute the gradient on the fly, making it very flexible as the user doesn't need to provide the derivative manually.
**Scenario**: Implied Volatility (Simplified).Find $x$ such that $x^2 - 10 = 6$ (i.e., solve $x^2 = 16$).

In [6]:
def model_price(volatility: float) -> float:
    # A dummy pricing model: Price = Volatility^2 - 10
    return volatility**2 - 10

target_price = 6.0
initial_guess = 2.0

# Solve for volatility
implied_vol = numerics.newton_solve(
    func=model_price, 
    target=target_price, 
    guess=initial_guess
)

print("\n--- Newton Solver ---")
if implied_vol:
    print_result("Root Found", implied_vol)
    print_result("Check Model", model_price(implied_vol))
else:
    print("Failed to converge.")


--- Newton Solver ---
Root Found: 4.000000
Check Model: 6.000000
